# Alpha - Beta Pruning and Tic Tac Toe

This notebook investigates the impact of alpha beta pruning on the game of Tic Tac Toe.

In [1]:
EMPTY = '_'
NOUGHT = 'o'
CROSS = 'x'

initial = [ [ EMPTY, EMPTY, EMPTY ],
            [ EMPTY, EMPTY, EMPTY ],
            [ EMPTY, EMPTY, EMPTY ]
          ]

In [2]:
WIN = +1
LOOSE = -1
DRAW = 0

def calcEmpty( state ):
    count = 0
    for i in range( len(state) ):
        for j in range( len(state[i])):
            if ( state[i][j] == EMPTY ):
                count = count + 1
    return count

def isComplete( state ):
    return calcEmpty(state) == 0

def isTerminal( state ):
    return ( evaluate( state ) is not None )

def isWin( player, state ):
    width, height = len(state[0]), len( state )
    # Check rows
    for i in range( height ):
        tmp = True
        for j in range( width ):
            if ( state[i][j] != player ):
                tmp = False
                break
        if ( tmp ):
            return True
    
    # Check columns
    for j in range( width ):
        tmp = True
        for i in range( height ):
            if ( state[i][j] != player ):
                tmp = False
                break
        if ( tmp ):
            return True
        
    # Check diagonal
    if ( width >= height ):
        # 3 rows, 5 columns
        limit = height
        for offset in range( width - limit + 1):
            tmp = True
            for d in range( limit ):
                if ( state[d][d+offset] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
            tmp = True
            for d in range( limit ):
                if ( state[height - 1 - d][d+offset] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
    else:
        # 5 rows, 3 columns
        limit = width
        for offset in range( height - limit + 1):
            tmp = True
            for d in range( limit ):
                if ( state[d+offset][d] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
            tmp = True
            for d in range( limit ):
                if ( state[d+offset][width-1-d] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
    return False

def evaluate( state ):
    result = None
    if ( isWin( CROSS, state ) ):
        result = WIN
    elif (isWin( NOUGHT, state ) ):
        result = LOOSE
    elif ( isComplete( state ) ):
        result = DRAW
    return result

In [3]:
def successor( player, state ):
    children = []
    #print('successor', state, 'isTerminal', isTerminal(state) )
    if ( not isTerminal( state ) ):
        for i in range( len(state) ):
            for j in range( len(state[i])):
                if ( state[i][j] == EMPTY ):
                    # print('found empty',i,j)
                    ns = [ [ c for c in r ] for r in state ]
                    ns[i][j] = player
                    children = children + [ ns ]
    return children

In [42]:
def heuristic( player, state ):
    winCount = 0
    for i in range( len( state ) ):
        for j in range( len( state[0] ) ):
            if ( state[i][j] == EMPTY ):
                ns = [ [ x for x in row ] for row in state ]
                ns[i][j] = player
                if ( isWin(player, ns ) ):
                    winCount = winCount + 1
    return winCount

In [38]:
def otherPlayer( player ):
    if ( player == CROSS ):
        other = NOUGHT
    elif ( player == NOUGHT ):
        other = CROSS
    return other

def alphaBetaSearch( player, state, alpha = None, beta = None, maxDepth = None, level = 0 ):
    tree = [ state, evaluate( state ), None, [] ]
    if ( isTerminal( state ) ):
        tree[2] = evaluate( state )
        return tree
    elif ( maxDepth is not None ) and ( level > maxDepth ):
        tree[2] = heuristic( player, state )
        return tree
    else:
        children = successor( player, state )
        nu = None
        if ( player == CROSS ):
            for c in children:
                childTree = alphaBetaSearch( otherPlayer( player ), c, alpha, beta, maxDepth, level + 1 )
                tree[3] = tree[3] + [ childTree ]
                if ( nu is None ) or ( childTree[2] > nu ):
                    nu = childTree[2]
                if ( alpha is None ) or ( nu > alpha ):
                    alpha = nu
                if ( beta is not None ) and ( alpha is not None) and ( beta <= alpha ):
                    break
            tree[2] = nu
            return tree
        elif ( player == NOUGHT ):
            for c in children:
                childTree = alphaBetaSearch( otherPlayer( player ), c, alpha, beta, maxDepth, level + 1 )
                tree[3] = tree[3] + [ childTree ]
                if ( nu is None ) or ( childTree[2] < nu ):
                    nu = childTree[2]
                if (beta is None ) or ( nu < beta ):
                    beta = nu
                if ( beta is not None ) and ( alpha is not None) and ( beta <= alpha ):
                    break
            tree[2] = nu
            return tree

In [46]:
def printState( state, level = 0 ):
    for i in range( len( state ) ):
        print( " " * ( level * 2 ), end = "" )
        for j in range( len( state[i] ) ):
            print( state[i][j], end="")
        print()
        
def printTree( tree, level = 0 ):
    state, value, h, children = tree
    print( " " * ( level * 2 ), end = "" )
    print( "Eval:", value, 'Value:', h, 'Heuristic:', heuristic( CROSS, state )  )
    printState( state, level )
    for c in children:
        printTree( c, level + 1 )

In [47]:
def countNumberOfNodes( tree ):
    _,_,_,children = tree
    numChildren = 0
    for c in children:
        numChildren = numChildren + countNumberOfNodes( c )
    return 1 + numChildren

def countNumberOfLeafNodes( tree ):
    _,_,_,children = tree
    if ( len(children) == 0 ):
        numLeaves = 1
    else:
        numLeaves = 0
        for c in children:
            numLeaves = numLeaves + countNumberOfLeafNodes( c )
    return numLeaves

def countNumberOfChildren( tree ):
    _,_,_,children = tree
    numChildren = 0
    for c in children:
        numChildren = numChildren + countNumberOfNodes( c )
    return numChildren

def calcAverageBranchingFactor( tree ):
    return countNumberOfChildren( tree ) / countNumberOfNodes( tree )
    
def calcDepthCount( tree, level = 0 ):
    _,_,_,children = tree
    if ( len(children) == 0 ):
        depthCount = level
    else:
        depthCount = 0
        for c in children:
            depthCount = depthCount + calcDepthCount( c, level + 1 )
    return depthCount
    
def calcAverageDepthOfLeafNodes( tree ):
    return calcDepthCount( tree, 0 ) / countNumberOfLeafNodes( tree )


In [48]:
state = [ [ CROSS, CROSS, NOUGHT ], [ EMPTY, NOUGHT, NOUGHT ],[ EMPTY, EMPTY, CROSS ] ]

printState( state, 3 )
print( successor( CROSS, state) )

      xxo
      _oo
      __x
[[['x', 'x', 'o'], ['x', 'o', 'o'], ['_', '_', 'x']], [['x', 'x', 'o'], ['_', 'o', 'o'], ['x', '_', 'x']], [['x', 'x', 'o'], ['_', 'o', 'o'], ['_', 'x', 'x']]]


In [49]:
tree = alphaBetaSearch( CROSS, state )
printTree( tree )

Eval: None Value: -1 Heuristic: 0
xxo
_oo
__x
  Eval: None Value: -1 Heuristic: 1
  xxo
  xoo
  __x
    Eval: -1 Value: -1 Heuristic: 0
    xxo
    xoo
    o_x
    Eval: None Value: 1 Heuristic: 1
    xxo
    xoo
    _ox
      Eval: 1 Value: 1 Heuristic: 0
      xxo
      xoo
      xox
  Eval: None Value: -1 Heuristic: 2
  xxo
  _oo
  x_x
    Eval: -1 Value: -1 Heuristic: 1
    xxo
    ooo
    x_x
  Eval: None Value: -1 Heuristic: 1
  xxo
  _oo
  _xx
    Eval: -1 Value: -1 Heuristic: 1
    xxo
    ooo
    _xx


In [50]:
tree = alphaBetaSearch( CROSS, initial )


In [51]:
numNodes = countNumberOfNodes( tree )
numLeafNodes = countNumberOfLeafNodes( tree )

b = calcAverageBranchingFactor( tree )

print('Number of Nodes', numNodes, 'Number of leaf nodes', numLeafNodes, 'b', b, 'Average depth of leaf nodes', calcAverageDepthOfLeafNodes( tree ) )


Number of Nodes 18297 Number of leaf nodes 7330 b 0.9999453462316227 Average depth of leaf nodes 8.08281036834925


In [52]:
18297/549946

0.03327053928931204